In [34]:
import re
import pandas as pd
from difflib import get_close_matches

# ------------------------------------------------------
# 1) 문자열 파싱
# ------------------------------------------------------
raw_input_data = """
3 - 호텔·관광경영학과, 11 - 경호학과, 6 - 치위생학과, 9 - 식품영양학과, 10 - 식품조리학과, 21 - 보건행정학과, 4 - 항공서비스과, 4 - 간호학과
"""

# 정규식으로 "숫자 - 전공명"을 추출
#  - (\d+)\s*-\s*([^,]+)
#    그룹1: 숫자(한 개 이상)
#    그룹2: 뒤에 콤마 전까지 모든 문자 (전공명)
pattern = re.compile(r'(\d+)\s*-\s*([^,]+)')

matches = pattern.findall(raw_input_data)

# 파싱된 (번호, 전공명) 튜플들을 저장
parsed_data = []
for match in matches:
    number_str, major_str = match
    number = int(number_str)
    major = major_str.strip()  # 앞뒤 공백 제거
    parsed_data.append((number, major))

# ------------------------------------------------------
# 2) 엑셀 파일 로드 및 컬럼명 설정
# ------------------------------------------------------
# 사용중인 엑셀 파일 경로와 엔진 설정
excel_file_path = 'C:\\Users\\user\\Desktop\\한국대학교육협의회_대학알리미 대학별 학과정보_20230614.xlsx'
df = pd.read_excel(excel_file_path, engine='openpyxl')

# 엑셀 내 실제 컬럼명에 맞춰 아래 두 값을 수정하세요.
MAJOR_COL = '학부_과(전공)명'       # 예: 학과(전공) 이름
SUBCLASS_COL = '표준분류소계열'   # 예: 표준분류소계열명

# 엑셀에 존재하는 전공명 리스트
unique_majors_in_file = df[MAJOR_COL].dropna().unique().tolist()

# ------------------------------------------------------
# 3) 전공명 유사도 매칭 함수
# ------------------------------------------------------
def get_standard_subclass(major_name: str) -> str:
    """
    difflib.get_close_matches를 활용해 major_name과 가장 유사한 전공(학과)명을 찾고,
    그에 대응되는 표준분류소계열명을 반환합니다.
    """
    match_candidates = get_close_matches(
        major_name,
        unique_majors_in_file,
        n=1,       # 가장 유사한 항목만
        cutoff=0.4 # 유사도 임계값(필요 시 조정)
    )
    if not match_candidates:
        return None

    best_match = match_candidates[0]
    # 매칭된 전공(학과)명과 동일한 행들 중 첫 번째 행에서 표준분류소계열명만 가져옴
    matched_row = df[df[MAJOR_COL] == best_match].iloc[0]
    return matched_row[SUBCLASS_COL]

# ------------------------------------------------------
# 4) 매칭 수행 및 결과 출력
# ------------------------------------------------------
results = []
for number, major in parsed_data:
    subclass_name = get_standard_subclass(major)
    results.append((number, major, subclass_name))

# (1) 매칭 결과를 한 줄씩 출력
print("매칭 결과:")
for number, major, subclass_name in results:
    print(f"{number} - {major} -> {subclass_name}")

# (2) 매칭된 표준소분류계열 전체 목록(중복 제거 후 정렬)
unique_subclasses = sorted({
    subclass_name
    for _, _, subclass_name in results
    if subclass_name is not None  # None을 제외(매칭 실패한 경우)
})

print("\n매칭된 표준소분류계열 전체 목록:")
print(", ".join(unique_subclasses))


매칭 결과:
3 - 호텔·관광경영학과 -> 경영학
11 - 경호학과 -> 체육
6 - 치위생학과 -> 임상보건
9 - 식품영양학과 -> 식품영양학
10 - 식품조리학과 -> 조리과학
21 - 보건행정학과 -> 보건관리
4 - 항공서비스과 -> 관광학
4 - 간호학과 -> 간호학

매칭된 표준소분류계열 전체 목록:
간호학, 경영학, 관광학, 보건관리, 식품영양학, 임상보건, 조리과학, 체육


In [7]:
import pandas as pd
import re

# 엑셀 파일 경로
excel_file_path = 'C:\\Users\\user\\Desktop\\한국대학교육협의회_대학알리미 대학별 학과정보_20230614.xlsx'

# ------------------------------------------------------
# 1. 엑셀 파일 읽기 
# ------------------------------------------------------
df = pd.read_excel(excel_file_path, engine='openpyxl')

# ------------------------------------------------------
# 2. 매핑 딕셔너리 생성
#    "표준분류소계열"을 key, "표준분류중계열"을 value로 매핑
# ------------------------------------------------------
sub_to_mid = dict(zip(df['표준분류소계열'], df['표준분류중계열']))

# ------------------------------------------------------
# 3. raw_input_data 문자열 파싱
#    콤마로 구분된 문자열을 리스트로 변환 (정규식 필요 없음)
# ------------------------------------------------------
raw_input_data = """
경영학, 공예, 공학교육, 광고・홍보학, 교육학, 국어・국문학, 국제지역학, 기타유럽어・문학, 독일어・문학, 디자인, 러시아어・문학, 무용, 문화・민속・미술사학, 미술학, 방송연예, 사회과교육, 사회복지학, 사회학, 순수미술, 스페인어・문학, 식품영양학, 실용음악, 언론・방송・매체학, 언어교육, 언어학, 역사・고고학, 연극, 영상예술, 영어・영문학, 예술・체육교육, 유아교육 , 음악학, 일본어・문학, 조리과학, 종교학, 주거학, 중국어・문학, 철학・윤리학, 체육, 초등교육, 축산학, 프랑스어・문학
"""
# 쉼표를 기준으로 분리하고, 좌우 공백 제거
input_subclasses = [s.strip() for s in raw_input_data.split(',') if s.strip()]

# ------------------------------------------------------
# 4. 매핑된 표준분류중계열 출력
# ------------------------------------------------------
print("입력된 표준분류소계열 -> 매핑된 표준분류중계열:")
matched_midclasses = []
for subclass in input_subclasses:
    # 매핑 정보가 없으면 "매핑없음"으로 표시
    midclass = sub_to_mid.get(subclass, "매핑없음")
    matched_midclasses.append(midclass)
    print(f"{subclass} -> {midclass}")

# ------------------------------------------------------
# 5. 유니크 표준분류중계열 전체 목록 (중복 제거 후 정렬) 출력
# ------------------------------------------------------
unique_midclasses = sorted({m for m in matched_midclasses if m != "매핑없음"})
print("\n유니크 표준분류중계열 목록:")
print(", ".join(unique_midclasses))


입력된 표준분류소계열 -> 매핑된 표준분류중계열:
경영학 -> 경영・경제
공예 -> 미술
공학교육 -> 교육
광고・홍보학 -> 경영・경제
교육학 -> 교육
국어・국문학 -> 언어・문학
국제지역학 -> 인문학
기타유럽어・문학 -> 언어・문학
독일어・문학 -> 언어・문학
디자인 -> 미술
러시아어・문학 -> 언어・문학
무용 -> 무용・체육
문화・민속・미술사학 -> 인문학
미술학 -> 미술
방송연예 -> 연극・영화
사회과교육 -> 교육
사회복지학 -> 사회과학
사회학 -> 사회과학
순수미술 -> 미술
스페인어・문학 -> 언어・문학
식품영양학 -> 생활과학
실용음악 -> 음악
언론・방송・매체학 -> 사회과학
언어교육 -> 교육
언어학 -> 언어・문학
역사・고고학 -> 인문학
연극 -> 연극・영화
영상예술 -> 응용예술
영어・영문학 -> 언어・문학
예술・체육교육 -> 교육
유아교육 -> 매핑없음
음악학 -> 음악
일본어・문학 -> 언어・문학
조리과학 -> 생활과학
종교학 -> 인문학
주거학 -> 생활과학
중국어・문학 -> 언어・문학
철학・윤리학 -> 인문학
체육 -> 무용・체육
초등교육 -> 교육
축산학 -> 농림・수산
프랑스어・문학 -> 언어・문학

유니크 표준분류중계열 목록:
경영・경제, 교육, 농림・수산, 무용・체육, 미술, 사회과학, 생활과학, 언어・문학, 연극・영화, 음악, 응용예술, 인문학
